# Codveda Internship – Level 1  
## Task 1: Data Cleaning and Preprocessing  
This notebook documents the complete data cleaning pipeline.


In [73]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [74]:
# Loading the raw dataset
stocks = pd.read_csv("2) Stock Prices Data Set.csv")
print("Initial size of the Dataset is :", stocks.shape)
print(stocks.head())

Initial size of the Dataset is : (497472, 7)
  symbol        date      open      high       low     close    volume
0    AAL  2014-01-02   25.0700   25.8200   25.0600   25.3600   8998943
1   AAPL  2014-01-02   79.3828   79.5756   78.8601   79.0185  58791957
2    AAP  2014-01-02  110.3600  111.8800  109.2900  109.7400    542711
3   ABBV  2014-01-02   52.1200   52.3300   51.5200   51.9800   4569061
4    ABC  2014-01-02   70.1100   70.2300   69.4800   69.8900   1148391


In [75]:
stocks.info() #Gives overall structure of the dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497472 entries, 0 to 497471
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   symbol  497472 non-null  object 
 1   date    497472 non-null  object 
 2   open    497461 non-null  float64
 3   high    497464 non-null  float64
 4   low     497464 non-null  float64
 5   close   497472 non-null  float64
 6   volume  497472 non-null  int64  
dtypes: float64(4), int64(1), object(2)
memory usage: 26.6+ MB


In [76]:
#Missing values detection
print("\nMissing Values:")
print(stocks.isnull().sum())

#Duplicate values detection
print("\nDuplicate Rows:", stocks.duplicated().sum())


Missing Values:
symbol     0
date       0
open      11
high       8
low        8
close      0
volume     0
dtype: int64

Duplicate Rows: 0


In [77]:
#Standarizing the column headers and removing inconsistencies like leading and trailing spaces
stocks.columns = stocks.columns.str.lower().str.strip()


In [78]:
#Converting the date column from object to datetime
stocks['date'] = pd.to_datetime(stocks['date'], errors='coerce')


# Convert numerical columns
num_cols = ['open', 'high', 'low', 'close', 'volume']
for col in num_cols:
    stocks[col] = pd.to_numeric(stocks[col], errors='coerce') #coerce does change the improper data to NaT for dates and NaN for numeric


stocks.info() #Rechecking the date column


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497472 entries, 0 to 497471
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   symbol  497472 non-null  object        
 1   date    497472 non-null  datetime64[ns]
 2   open    497461 non-null  float64       
 3   high    497464 non-null  float64       
 4   low     497464 non-null  float64       
 5   close   497472 non-null  float64       
 6   volume  497472 non-null  int64         
dtypes: datetime64[ns](1), float64(4), int64(1), object(1)
memory usage: 26.6+ MB


In [79]:
#Diagnostic step for the Missing values
print(stocks.isnull().sum())#shows the mising values
len(stocks)#calculates the no. of rows

stocks[['open','high','low']].isna().mean() * 100 
#This calculates the %of the missing values
# If percentage is less than one its better to drop the rows otherwise investigate it




symbol     0
date       0
open      11
high       8
low        8
close      0
volume     0
dtype: int64


open    0.002211
high    0.001608
low     0.001608
dtype: float64

In [80]:
#Investigating the missing values
stocks[stocks[['open','high','low']].isna().any(axis=1)]


,symbol,date,open,high,low,close,volume
166348,VRTX,2015-05-12,NaN,NaN,NaN,124.0800,569747
175557,REGN,2015-06-09,NaN,NaN,NaN,526.0900,12135
182011,WRK,2015-06-26,NaN,NaN,NaN,61.9000,100
188547,DHR,2015-07-17,NaN,88.76,88.24,88.7200,2056819
188578,ES,2015-07-17,NaN,48.49,47.85,47.9200,1246786
188760,O,2015-07-17,NaN,47.31,46.83,46.9900,1229513
249223,DHR,2016-01-12,NaN,NaN,NaN,88.5500,0
249438,O,2016-01-12,NaN,NaN,NaN,52.4300,0
278801,UA,2016-04-07,NaN,NaN,NaN,41.5600,0
308365,FTV,2016-07-01,NaN,NaN,NaN,49.5400,0


In [81]:
stocks = stocks.dropna(subset=['open', 'high','low']) #Dropping missing values as per diagnostic identifiers and %missing values
stocks[stocks[['open','high','low']].isna().any(axis=1)]#Rechecking for the missing values if any left

,symbol,date,open,high,low,close,volume


In [82]:
# Checking whether there are any inconsistencies with the symbol data
print(stocks['symbol'].nunique())
print(stocks['symbol'].str.upper().nunique())
#Cross check both the numbers to detect inconsistencies in the format

505
505


In [83]:
#Diagnosing data as per stock rules and advanced cleaning Stock rules:

#High >= Open
#High >= Close
#Low <= Open
#Low <= Close
#if this violates that is considered as an error

invalid_rows = stocks[
    (stocks['high'] < stocks['low']) |
    (stocks['high'] < stocks['open']) |
    (stocks['high'] < stocks['close']) |
    (stocks['low'] > stocks['open']) |
    (stocks['low'] > stocks['close'])
]

print("Invalid Logical Rows:", invalid_rows.shape[0])

#Removing Invalid rows
stocks = stocks.drop(invalid_rows.index)


Invalid Logical Rows: 12


In [84]:
#Final checks for the data
print("\nFinal Size of the dataset is:", stocks.shape)
print("\nRemaining Missing Values:")
print(stocks.isnull().sum())
print("\nFinal Data Types:")
print(stocks.dtypes)



Final Size of the dataset is: (497449, 7)

Remaining Missing Values:
symbol    0
date      0
open      0
high      0
low       0
close     0
volume    0
dtype: int64

Final Data Types:
symbol            object
date      datetime64[ns]
open             float64
high             float64
low              float64
close            float64
volume             int64
dtype: object


In [85]:
#Saving the cleaned dataset as .csv
stocks.to_csv("cleaned_stocks_data.csv", index=False)
